# Peer-graded Assignment: Capstone Project - The Battle of Neighborhoods
# *Choosing the right hotel to stay in*

Author: Radoslaw Cholewinski

Date: May 2020

Below please find my final report. So far two sections are available::
1. Introduction/Business Problem
2. Data

## 1. Introduction/Business Problem

As the coronavirus outbreak has dominated everyone's life on the planet, I am wondering now when people could travel again. I made couple of plans for this year I need to postpone, but I thought I could use that travel-free time to build some tools for more efficient travel choices in the future. As the capstone project, I decided to develop a simple model which groups hotels in a given city into clusters based on facilities and services available in the neighborhood of each hotel. Travel search engines provide users with (sometimes very long) lists of possible accommodation options, but in my opinion they still fail to properly cluster hotels as e.g. 'close to restaurants' or 'close to sightseeings' or 'located in a remote area', which would facilitate the search process.

I decided to develop my model for Naples, Italy, where I planned to be right now instead of sitting on my balcony. The model can be however easily generalized to any place in the world. I haven't been in Naples yet and have limited knowledge of Naples boroughs. As there are numerous hotels there, I would like to divide them into groups, see in what terms the groups differ and narrow my search to a possibly limited number of hotels residing in a group of my choice. In my case it would be a proximity of pizza places, Italian restaurants and ice-cream shops.

Other, more commercial, applications of this model are possible. A hotel search engine could add a new filter to cluster the results as described in the first paragraph, so users find their desired accommodation easier and faster. Or a booking website could use clusters to identify hotels which are most attractive to prospective clients and discriminate the commission the hotel pays to the website on that basis. The booking website could also target its users by recommending them hotels basing on the cluster hotels belong to and the reservations users made before. To add more, local authorities could differentiate tourist tax levels basing on the attractiveness of each cluster.   



## 2. Data

In this project data provided by Foursquare API is used. Foursquare lists *inter alia* hotels and provides recommended venues near a given hotel. A Foursquare 'explore' endpoint is used to get a list of recommended hotels in Naples. The total number of results is 98. While in theory Foursquare API returns up to 50 results only, the query surprisingly retrieved 89 results, without any need for further experimentation with the API. In the author's opinion, 89 options are pretty sufficient to choose a desired hotel.

Next, for each hotel an 'explore' query is performed to find all the venues within a walking distance, set to 500 meters. Foursquare API enables to retrieve, apart from the location data, the venue category data easily. 3524 venues are found, which means an average hotel is located within a walking distance to approximately 40 venues. 133 distinct venue categories are returned. 

The next sections describes the clustering process, discusses the results and major findings.